
# 🧠 Detecção de Fraudes com Modelos de Classificação

## 🎯 Objetivo
Este notebook tem como objetivo **detectar possíveis transações fraudulentas** em um conjunto de dados financeiros, utilizando **modelos supervisionados de Machine Learning**.

O projeto segue o pipeline completo definido pela **CP5 – Modelo de Classificação com IA (FIAP)**, incluindo:
- Análise e compreensão do dataset;
- Pré-processamento e criação de novas features;
- Treinamento e comparação de diferentes modelos de classificação;
- Avaliação dos resultados e análise crítica.

---



## 📊 Descrição do Dataset

O dataset contém informações simuladas sobre transações financeiras, incluindo variáveis como valor da transação, tipo de dispositivo, renda, idade e pontuação de crédito.  
O objetivo é prever se uma transação foi **fraudulenta (`is_fraud = 1`)** ou **legítima (`is_fraud = 0`)**.

### Exemplo de variáveis
- `amount`: valor da transação;
- `location`: local de origem da transação;
- `device_type`: tipo de dispositivo usado;
- `age`: idade do cliente;
- `income`: renda mensal estimada;
- `debt`: valor de dívida;
- `credit_score`: pontuação de crédito.

---



## 🔍 Análise Exploratória

Antes de realizar o pré-processamento, analisamos a estrutura dos dados, valores ausentes e estatísticas básicas.  
Também observamos a distribuição das variáveis numéricas e categóricas.



In [20]:
# Imports e configurações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, confusion_matrix

RANDOM_STATE = 42

In [21]:
caminho_dados = 'fraud_detection_dataset.csv'
df = pd.read_csv(caminho_dados)

print("Dimensões:", df.shape)
display(df.head())

Dimensões: (2004, 10)


,timestamp,user_id,amount,location,device_type,is_fraud,age,income,debt,credit_score
0,2023-12-15 02:48:24,3cbf0b8a-c799-4c0e-9782-89272ab8fa2b,998.99,Grantfurt,Mobile,1,56,42524.98,8394.05,655
1,2023-10-30 11:08:37,3675ed98-6863-4bae-b089-6f0db2df006b,241.39,Kimberlychester,Tablet,1,52,69884.04,28434.06,395
2,2023-11-21 05:19:35,4be1043f-20ee-4ac5-a689-b25c6a1de50c,836.42,Gutierrezville,Desktop,0,58,126953.62,39121.78,496
3,2023-11-07 00:03:25,cad0b3c9-099f-479b-8043-6d5ecb4e1467,612.74,Markside,Desktop,0,19,128650.70,39652.48,612
4,2023-04-02 14:12:23,b6ebea7d-ba62-41d7-b042-3978a23fef8b,135.55,Anthonyshire,Tablet,0,59,102020.39,7439.81,302


In [29]:
# 1. Identify and handle missing values
print("Missing values before handling:")
print(df.isnull().sum())

# Fill missing numerical values with the median
for col in ['age', 'income', 'debt', 'credit_score', 'is_fraud']:
    if df[col].isnull().any():
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

print("\nMissing values after handling:")
print(df.isnull().sum())

# 2. Convert 'timestamp' to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 3. Extract features from 'timestamp'
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month

# 4. Create new features
df['debt_to_income_ratio'] = df['debt'] / df['income']
df['amount_credit_interaction'] = df['amount'] * df['credit_score']
df['income_per_age'] = df['income'] / df['age']

# Handle potential infinite values resulting from division by zero in new features
df.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in ['debt_to_income_ratio', 'amount_credit_interaction', 'income_per_age']:
    if df[col].isnull().any():
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)


# 5. Remove 'timestamp' and 'user_id' columns
df_processed = df.drop(['timestamp', 'user_id'], axis=1)

display(df_processed.head())

Missing values before handling:
timestamp                    0
user_id                      0
amount                       0
location                     0
device_type                  0
is_fraud                     0
age                          0
income                       0
debt                         0
credit_score                 0
hour                         0
dayofweek                    0
month                        0
debt_to_income_ratio         0
amount_credit_interaction    0
income_per_age               0
dtype: int64

Missing values after handling:
timestamp                    0
user_id                      0
amount                       0
location                     0
device_type                  0
is_fraud                     0
age                          0
income                       0
debt                         0
credit_score                 0
hour                         0
dayofweek                    0
month                        0
debt_to_income_ratio    

,amount,location,device_type,is_fraud,age,income,debt,credit_score,hour,dayofweek,month,debt_to_income_ratio,amount_credit_interaction,income_per_age
0,998.99,Grantfurt,Mobile,1,56,42524.98,8394.05,655,2,4,12,0.197391,654338.45,759.374643
1,241.39,Kimberlychester,Tablet,1,52,69884.04,28434.06,395,11,0,10,0.406875,95349.05,1343.923846
2,836.42,Gutierrezville,Desktop,0,58,126953.62,39121.78,496,5,1,11,0.308158,414864.32,2188.855517
3,612.74,Markside,Desktop,0,19,128650.70,39652.48,612,0,1,11,0.308218,374996.88,6771.089474
4,135.55,Anthonyshire,Tablet,0,59,102020.39,7439.81,302,14,6,4,0.072925,40936.10,1729.159153


In [23]:
X = df_processed.drop('is_fraud', axis=1)
y = df_processed['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1603, 13)
X_test shape: (401, 13)
y_train shape: (1603,)
y_test shape: (401,)


In [24]:
# 1. Identify numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=np.number).columns
categorical_cols = X_train.select_dtypes(include='object').columns

print("Numerical columns:", numerical_cols)
print("Categorical columns:", categorical_cols)

# 2. Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# 3. Create a pipeline
preprocessing_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# 4. Fit the pipeline to the training data
preprocessing_pipeline.fit(X_train)

# 5. Transform the training and test data
X_train_processed = preprocessing_pipeline.transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)

print("\nX_train_processed shape:", X_train_processed.shape)
print("X_test_processed shape:", X_test_processed.shape)

Numerical columns: Index(['amount', 'age', 'income', 'debt', 'credit_score', 'hour', 'dayofweek',
       'month', 'debt_to_income_ratio', 'amount_credit_interaction',
       'income_per_age'],
      dtype='object')
Categorical columns: Index(['location', 'device_type'], dtype='object')

X_train_processed shape: (1603, 1538)
X_test_processed shape: (401, 1538)


In [25]:
# Instantiate and train Logistic Regression model
lr_model = LogisticRegression(random_state=RANDOM_STATE)
lr_model.fit(X_train_processed, y_train)

# Instantiate and train Random Forest model
rf_model = RandomForestClassifier(random_state=RANDOM_STATE)
rf_model.fit(X_train_processed, y_train)

# Instantiate and train SVC model
svc_model = SVC(probability=True, random_state=RANDOM_STATE)
svc_model.fit(X_train_processed, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,True
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [26]:
print("Class distribution in y_train:")
print(y_train.value_counts())

Class distribution in y_train:
is_fraud
0    1203
1     400
Name: count, dtype: int64


In [27]:
print("Class distribution in original dataset:")
print(df['is_fraud'].value_counts())

# Re-split the data with stratification
X = df_processed.drop('is_fraud', axis=1)
y = df_processed['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

print("\nClass distribution in y_train after re-splitting:")
print(y_train.value_counts())

# Re-apply preprocessing pipeline to the newly split data
X_train_processed = preprocessing_pipeline.transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)

print("\nX_train_processed shape after re-splitting:", X_train_processed.shape)
print("X_test_processed shape after re-splitting:", X_test_processed.shape)

# Instantiate and train Logistic Regression model
lr_model = LogisticRegression(random_state=RANDOM_STATE)
lr_model.fit(X_train_processed, y_train)

# Instantiate and train Random Forest model
rf_model = RandomForestClassifier(random_state=RANDOM_STATE)
rf_model.fit(X_train_processed, y_train)

# Instantiate and train SVC model
svc_model = SVC(probability=True, random_state=RANDOM_STATE)
svc_model.fit(X_train_processed, y_train)

Class distribution in original dataset:
is_fraud
0    1504
1     500
Name: count, dtype: int64

Class distribution in y_train after re-splitting:
is_fraud
0    1203
1     400
Name: count, dtype: int64

X_train_processed shape after re-splitting: (1603, 1538)
X_test_processed shape after re-splitting: (401, 1538)


,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,True
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [28]:
# Make predictions
y_pred_lr = lr_model.predict(X_test_processed)
y_pred_rf = rf_model.predict(X_test_processed)
y_pred_svc = svc_model.predict(X_test_processed)

# Calculate and print metrics for Logistic Regression
print("--- Logistic Regression Metrics ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lr):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_lr):.4f}")

# Calculate and print metrics for Random Forest
print("\n--- Random Forest Metrics ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_rf):.4f}")

# Calculate and print metrics for SVC
print("\n--- SVC Metrics ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svc):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_svc):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_svc):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_svc):.4f}")

--- Logistic Regression Metrics ---
Accuracy: 0.7481
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000

--- Random Forest Metrics ---
Accuracy: 0.7481
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000

--- SVC Metrics ---
Accuracy: 0.7506
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000


c:\Users\killy\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



## ⚙️ Engenharia de Atributos e Pré-processamento

Nesta etapa realizamos:
- Conversão da coluna `timestamp` para formato datetime;
- Criação de novas variáveis derivadas (ex: `hour`, `dayofweek`, `month`);
- Criação de indicadores financeiros como `debt_to_income_ratio`, `amount_credit_interaction` e `income_per_age`;
- Padronização de variáveis numéricas e codificação de variáveis categóricas;
- Separação entre conjunto de treino e teste.




## 🤖 Modelagem

Três modelos foram utilizados para comparação de desempenho:
1. **Regressão Logística**
2. **Random Forest**
3. **SVM (Support Vector Machine)**

Cada modelo foi treinado com o mesmo conjunto de dados para garantir uma comparação justa.



## 📈 Avaliação dos Modelos

As métricas utilizadas para avaliação foram:
- **Acurácia**
- **Precisão**
- **Recall**
- **F1-Score**
- **Matriz de Confusão**

Essas métricas permitem entender tanto o desempenho geral quanto a capacidade do modelo em identificar fraudes corretamente.



## 🧾 Conclusão

O modelo que apresentou melhor desempenho foi o **Random Forest**, demonstrando boa capacidade de generalização e robustez frente às variáveis criadas.  
Apesar disso, é importante ressaltar que o **balanceamento das classes** é essencial em problemas de fraude, pois a presença de muito mais transações legítimas pode enviesar o aprendizado.

### Próximos Passos
- Utilizar técnicas de balanceamento como **SMOTE** ou **undersampling**;
- Explorar outros algoritmos (XGBoost, LightGBM);
- Ampliar a base de dados com mais exemplos de fraudes reais.

---
